In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.4/647.4 kB 31.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.2-cp310-cp310-linux_x86_64.whl size=582716 sha256=e64bf85f2ad45167a82a9e8be570ae475f87241ee8a0b64f30a07370a2faeb58
  Stored in directory: /root/.cache/pip/wheels/7a/d9/59/473fa56df8e39430eeda369500b4e7127f5b243ba24c3c4297
Successfully built annoy


In [4]:
from collections import defaultdict, Counter
from typing import List, Dict
import pickle
import time

from tqdm import tqdm
import numpy as np
import pandas as pd
from annoy import AnnoyIndex
import polars as pl
import networkx as nx
import scipy.sparse
import scipy.sparse as sp
from scipy import linalg
from scipy.special import iv
from sklearn import preprocessing
from sklearn.utils.extmath import randomized_svd

In [5]:
LOCALES = ["FR", "ES", "IT"]
VER = "07"
DIR = "/gdrive/MyDrive/amazon_kdd_2023/"
TOP_N = 50

# hyper-parameter for proNE
EMB_DIM = 1024
N_EPOCH = 10
MU = 0
THETA = 0.5

# functions

In [6]:
def preprocess(df:pl.DataFrame) -> pl.DataFrame:
    df = df.explode(["prev_items"])
    df = df.with_columns(
        df.select(pl.col("session_id").cumcount().over("session_id").alias("sequence_num"))
    )
    return df

In [7]:
def build_graph(df:pl.DataFrame):
    df = df.sort(["session_id", "sequence_num"], descending=[False, False])
    df = df.with_columns(
        pl.col("item").shift().over("session_id").alias("prev_item")
    )
    df = df.filter(
        (pl.col("prev_item").is_not_null()) &
        (pl.col("prev_item") != pl.col("item"))
    )
    df = df[["item", "prev_item"]]
    # df = df.unique()
    return df

In [8]:
def convert_item_id(df:pl.DataFrame):
    unique_item_ids = sorted(list(set(df["item"].unique().to_list() + df["prev_item"].unique().to_list())))
    item_id2index = dict(zip(unique_item_ids, range(len(unique_item_ids))))
    df = df.with_columns([
        pl.col("item").map_dict(item_id2index).alias("item"),
        pl.col("prev_item").map_dict(item_id2index).alias("prev_item"),
    ])
    return df, unique_item_ids, item_id2index

In [9]:
# https://github.com/THUDM/ProNE/blob/master/proNE.py
class ProNE():
	def __init__(self, graph_file, emb_file1, emb_file2, dimension):
		self.graph = graph_file
		self.emb1 = emb_file1
		self.emb2 = emb_file2
		self.dimension = dimension

		self.G = nx.read_edgelist(self.graph, nodetype=int, create_using=nx.DiGraph())
		self.G = self.G.to_undirected()
		self.node_number = self.G.number_of_nodes()
		matrix0 = scipy.sparse.lil_matrix((self.node_number, self.node_number))

		for e in self.G.edges():
			if e[0] != e[1]:
				matrix0[e[0], e[1]] = 1
				matrix0[e[1], e[0]] = 1
		self.matrix0 = scipy.sparse.csr_matrix(matrix0)
		print(matrix0.shape)

	def get_embedding_rand(self, matrix):
		# Sparse randomized tSVD for fast embedding
		t1 = time.time()
		l = matrix.shape[0]
		smat = scipy.sparse.csc_matrix(matrix)  # convert to sparse CSC format
		print('svd sparse', smat.data.shape[0] * 1.0 / l ** 2)
		U, Sigma, VT = randomized_svd(smat, n_components=self.dimension, n_iter=5, random_state=None)
		U = U * np.sqrt(Sigma)
		U = preprocessing.normalize(U, "l2")
		print('sparsesvd time', time.time() - t1)
		return U

	def get_embedding_dense(self, matrix, dimension):
		# get dense embedding via SVD
		t1 = time.time()
		U, s, Vh = linalg.svd(matrix, full_matrices=False, check_finite=False, overwrite_a=True)
		U = np.array(U)
		U = U[:, :dimension]
		s = s[:dimension]
		s = np.sqrt(s)
		U = U * s
		U = preprocessing.normalize(U, "l2")
		print('densesvd time', time.time() - t1)
		return U

	def pre_factorization(self, tran, mask):
		# Network Embedding as Sparse Matrix Factorization
		t1 = time.time()
		l1 = 0.75
		C1 = preprocessing.normalize(tran, "l1")
		neg = np.array(C1.sum(axis=0))[0] ** l1

		neg = neg / neg.sum()

		neg = scipy.sparse.diags(neg, format="csr")
		neg = mask.dot(neg)
		print("neg", time.time() - t1)

		C1.data[C1.data <= 0] = 1
		neg.data[neg.data <= 0] = 1

		C1.data = np.log(C1.data)
		neg.data = np.log(neg.data)

		C1 -= neg
		F = C1
		features_matrix = self.get_embedding_rand(F)
		return features_matrix

	def chebyshev_gaussian(self, A, a, order=10, mu=0.5, s=0.5):
		# NE Enhancement via Spectral Propagation
		print('Chebyshev Series -----------------')
		t1 = time.time()

		if order == 1:
			return a

		A = sp.eye(self.node_number) + A
		DA = preprocessing.normalize(A, norm='l1')
		L = sp.eye(self.node_number) - DA

		M = L - mu * sp.eye(self.node_number)

		Lx0 = a
		Lx1 = M.dot(a)
		Lx1 = 0.5 * M.dot(Lx1) - a

		conv = iv(0, s) * Lx0
		conv -= 2 * iv(1, s) * Lx1
		for i in range(2, order):
			Lx2 = M.dot(Lx1)
			Lx2 = (M.dot(Lx2) - 2 * Lx1) - Lx0
			#         Lx2 = 2*L.dot(Lx1) - Lx0
			if i % 2 == 0:
				conv += 2 * iv(i, s) * Lx2
			else:
				conv -= 2 * iv(i, s) * Lx2
			Lx0 = Lx1
			Lx1 = Lx2
			del Lx2
			print('Bessell time', i, time.time() - t1)
		mm = A.dot(a - conv)
		emb = self.get_embedding_dense(mm, self.dimension)
		return emb

In [10]:
# https://github.com/THUDM/ProNE/blob/master/proNE.py
def save_embedding(emb_file, features):
	# save node embedding into emb_file with word2vec format
	f_emb = open(emb_file, 'w')
	f_emb.write(str(len(features)) + " " + str(features.shape[1]) + "\n")
	for i in range(len(features)):
		s = str(i) + " " + " ".join(str(f) for f in features[i].tolist())
		f_emb.write(s + "\n")
	f_emb.close()

In [11]:
def generate_annoy_index(matrix):
    index = AnnoyIndex(EMB_DIM, 'angular')

    for idx,idx_embedding in enumerate(matrix):
        index.add_item(idx, idx_embedding)
        
    index.build(50)
    
    return index

In [12]:
def make_nns_matrix(annoy_index, item_ids, item_id2index, k=100):
    aid_xs = []
    aid_ys = []
    dists = []

    for item_id in tqdm(item_ids):
        item_index = item_id2index[item_id]
        nns = annoy_index.get_nns_by_item(item_index, k+1, include_distances=True)
        aid_y = [item_ids[idx] for idx in list(nns[0][1:])]
        dist = list(nns[1][1:])
        aid_xs.extend([item_id] * k)
        aid_ys.extend(aid_y)
        dists.extend(dist)
    df = pl.DataFrame({"item": aid_xs, 'candidate_item': aid_ys, 'prone_distance': dists})

    # rank付与
    df = df.with_columns(
        pl.col("prone_distance").rank(descending=False, method="min").over("item").alias("prone_rank")
    )

    return df

# for local train

In [13]:
train = pl.read_parquet(DIR + "data/preprocessed/task2/train_task2.parquet")
test2_1 = pl.read_parquet(DIR + "data/preprocessed/task2/test_task2_phase1.parquet")
test2_2 = pl.read_parquet(DIR + "data/preprocessed/task2/test_task2_phase2.parquet")
test3_1 = pl.read_parquet(DIR + "data/preprocessed/task3/test_task3_phase1.parquet").filter(pl.col("locale").is_in(LOCALES))
test3_2 = pl.read_parquet(DIR + "data/preprocessed/task3/test_task3_phase2.parquet").filter(pl.col("locale").is_in(LOCALES))
test3_1 = test3_1.with_columns(
    (pl.col("session_id") + "_from_task3").alias("session_id")
)
test3_2 = test3_2.with_columns(
    (pl.col("session_id") + "_from_task3").alias("session_id")
)
test = pl.concat([test2_1, test2_2, test3_1, test3_2])

In [14]:
train = preprocess(train)
test = preprocess(test)

In [15]:
df = pl.concat([
    train["prev_items", "locale", "session_id", "sequence_num"],
    test["prev_items", "locale", "session_id", "sequence_num"],
])
df = df.rename({"prev_items":"item"})

In [16]:
candidates = []
for locale in LOCALES:
    # filter by loacle
    df_by_locale = df.filter(pl.col("locale") == locale)

    # build graph
    graph_df = build_graph(df_by_locale)
    graph_df, item_ids, item_id2index = convert_item_id(graph_df)
    graph_df.write_csv(DIR + f"data/interim/graph/task2/edge_list_{VER}_{locale}_task2_for_local_train_or_eval.txt", has_header=False, separator=" ")
    item_ids_name = f"item_id2index_{VER}_{locale}_for_train_or_eval.pickle"
    with open(DIR + "data/interim/graph/task2/graph_" + item_ids_name, "wb") as f:
        pickle.dump(item_id2index, f)

    # generate graph embedding by proNE 
    model = ProNE(
        DIR + f"data/interim/graph/task2/edge_list_{VER}_{locale}_task2_for_local_train_or_eval.txt",
        "",
        "",
        EMB_DIM,
    )
    features_matrix = model.pre_factorization(model.matrix0, model.matrix0)
    embeddings_matrix = model.chebyshev_gaussian(model.matrix0, features_matrix, N_EPOCH, MU, THETA)
    np.save(DIR + f"models/task2/graph_embedding_{VER}_{locale}_for_local_train_or_eval.npy", embeddings_matrix)

    # generate_candidates
    annoy_index = generate_annoy_index(embeddings_matrix)
    candidate = make_nns_matrix(annoy_index, item_ids, item_id2index, k=TOP_N)
    candidate = candidate.with_columns(pl.lit(locale).alias("locale"))
    candidates.append(candidate)

candidate = pl.concat(candidates)
candidate.write_parquet(DIR + f"data/interim/candidates/task2/prone_{VER}_for_local_or_eval.parquet")

(43256, 43256)
neg 0.03377246856689453
svd sparse 0.0001637138703505436
sparsesvd time 50.9586181640625
Chebyshev Series -----------------
Bessell time 2 4.029489040374756
Bessell time 3 6.129669189453125
Bessell time 4 8.171926736831665
Bessell time 5 10.233439445495605
Bessell time 6 12.287846088409424
Bessell time 7 14.354411602020264
Bessell time 8 16.414438486099243
Bessell time 9 18.466026306152344
densesvd time 6.584527015686035


100%|██████████| 43256/43256 [01:07<00:00, 645.50it/s]


(40075, 40075)
neg 0.013020753860473633
svd sparse 0.000148673202810233
sparsesvd time 44.95578980445862
Chebyshev Series -----------------
Bessell time 2 3.5795812606811523
Bessell time 3 5.37935471534729
Bessell time 4 7.168367385864258
Bessell time 5 8.941052436828613
Bessell time 6 10.730618953704834
Bessell time 7 12.541410684585571
Bessell time 8 14.330695629119873
Bessell time 9 16.120096921920776
densesvd time 6.163109064102173


100%|██████████| 40075/40075 [01:03<00:00, 633.34it/s]


(48716, 48716)
neg 0.015516281127929688
svd sparse 0.00014724205115066545
sparsesvd time 53.98737573623657
Chebyshev Series -----------------
Bessell time 2 4.66695761680603
Bessell time 3 7.212962865829468
Bessell time 4 9.781599044799805
Bessell time 5 12.242364883422852
Bessell time 6 14.787699699401855
Bessell time 7 17.351515769958496
Bessell time 8 19.834309101104736
Bessell time 9 22.37743330001831
densesvd time 8.828718662261963


100%|██████████| 48716/48716 [01:23<00:00, 582.29it/s]


In [17]:
candidate.head()

item,candidate_item,prone_distance,prone_rank,locale
str,str,f64,u32,str
"""0008402787""","""274994922X""",0.042193,1,"""FR"""
"""0008402787""","""2290210781""",0.04322,2,"""FR"""
"""0008402787""","""2290374423""",0.047929,3,"""FR"""
"""0008402787""","""B07KJFK2H8""",0.048319,4,"""FR"""
"""0008402787""","""2364808731""",0.060309,5,"""FR"""


# MRR@100

In [18]:
train = pl.read_parquet(DIR + "data/preprocessed/task2/train_task2.parquet")

In [19]:
candidate = pl.read_parquet(DIR + f"data/interim/candidates/task2/prone_{VER}_for_local_or_eval.parquet")

In [20]:
# last_itemの抽出
last_item_list = []
prev_items_list = train["prev_items"].to_list()
for prev_items in prev_items_list:
    last_item_list.append(prev_items[-1])
train = train.with_columns(pl.Series(name="last_item", values=last_item_list))

In [21]:
train = train.join(candidate, left_on=["locale", "last_item"], right_on=["locale", "item"], how="left")
train = train.filter(~pl.col("candidate_item").is_in(pl.col("prev_items")))
train = train.sort(["session_id", "prone_distance"], descending=[False, False])
train = train.with_columns((pl.col("candidate_item") == pl.col("next_item")).cast(pl.Int8).alias("label"))
label_lists = train.groupby("session_id", maintain_order=True).all()["label"].to_list()

In [22]:
# MRRの計算
rr = 0
for labels in label_lists:
    labels = labels[:100]
    for i, label in enumerate(labels):
        if label == 1:
            rr += 1 / (i+1)
            break
mrr = rr / len(label_lists)
print("MRR:", round(mrr, 5))

MRR: 0.1994


# for inference

In [23]:
train = pl.read_parquet(DIR + "data/preprocessed/task2/train_task2.parquet")
test2_1 = pl.read_parquet(DIR + "data/preprocessed/task2/test_task2_phase1.parquet")
test2_2 = pl.read_parquet(DIR + "data/preprocessed/task2/test_task2_phase2.parquet")
test3_1 = pl.read_parquet(DIR + "data/preprocessed/task3/test_task3_phase1.parquet").filter(pl.col("locale").is_in(LOCALES))
test3_2 = pl.read_parquet(DIR + "data/preprocessed/task3/test_task3_phase2.parquet").filter(pl.col("locale").is_in(LOCALES))
test3_1 = test3_1.with_columns(
    (pl.col("session_id") + "_from_task3").alias("session_id")
)
test3_2 = test3_2.with_columns(
    (pl.col("session_id") + "_from_task3").alias("session_id")
)
test = pl.concat([test2_1, test2_2, test3_1, test3_2])

In [24]:
# trainのnext_itemをprev_itemsにappendする
prev_items_list = train["prev_items"].to_list()
next_item_list = train["next_item"].to_list()
prev_items_list_updated = []
for prev_items, next_item in zip(prev_items_list, next_item_list):
    prev_items.append(next_item)
    prev_items_list_updated.append(prev_items)

train = train.with_columns(
    pl.Series(name="prev_items", values=prev_items_list_updated)
)

In [25]:
train = preprocess(train)
test = preprocess(test)
df = pl.concat([
    train["prev_items", "locale", "session_id", "sequence_num"],
    test["prev_items", "locale", "session_id", "sequence_num"],
])
df = df.rename({"prev_items":"item"})

In [26]:
candidates = []
for locale in LOCALES:
    # filter by loacle
    df_by_locale = df.filter(pl.col("locale") == locale)

    # build graph
    graph_df = build_graph(df_by_locale)
    graph_df, item_ids, item_id2index = convert_item_id(graph_df)
    graph_df.write_csv(DIR + f"data/interim/graph/task2/edge_list_{VER}_{locale}_task2_for_inference.txt", has_header=False, separator=" ")
    item_ids_name = f"item_id2index_{VER}_{locale}_for_inference.pickle"
    with open(DIR + "data/interim/graph/task2/graph_" + item_ids_name, "wb") as f:
        pickle.dump(item_id2index, f)

    # generate graph embedding by proNE 
    model = ProNE(
        DIR + f"data/interim/graph/task2/edge_list_{VER}_{locale}_task2_for_inference.txt",
        "",
        "",
        EMB_DIM,
    )
    features_matrix = model.pre_factorization(model.matrix0, model.matrix0)
    embeddings_matrix = model.chebyshev_gaussian(model.matrix0, features_matrix, N_EPOCH, MU, THETA)
    np.save(DIR + f"models/task2/graph_embedding_{VER}_{locale}_for_inference.npy", embeddings_matrix)

    # generate_candidates
    annoy_index = generate_annoy_index(embeddings_matrix)
    candidate = make_nns_matrix(annoy_index, item_ids, item_id2index, k=TOP_N)
    candidate = candidate.with_columns(pl.lit(locale).alias("locale"))
    candidates.append(candidate)

candidate = pl.concat(candidates)
candidate.write_parquet(DIR + f"data/interim/candidates/task2/prone_{VER}_for_inference.parquet")

(44507, 44507)
neg 0.01949334144592285
svd sparse 0.00020715310363132717
sparsesvd time 63.08721351623535
Chebyshev Series -----------------
Bessell time 2 5.553625822067261
Bessell time 3 8.453381299972534
Bessell time 4 11.316065311431885
Bessell time 5 14.175156831741333
Bessell time 6 17.11068296432495
Bessell time 7 20.01768660545349
Bessell time 8 22.876051902770996
Bessell time 9 25.75657033920288
densesvd time 9.396045446395874


100%|██████████| 44507/44507 [01:38<00:00, 453.08it/s]


(42439, 42439)
neg 0.018636465072631836
svd sparse 0.00017864504349470627
sparsesvd time 51.37723898887634
Chebyshev Series -----------------
Bessell time 2 4.699989557266235
Bessell time 3 7.143524408340454
Bessell time 4 9.577183723449707
Bessell time 5 12.0259530544281
Bessell time 6 14.480803489685059
Bessell time 7 16.914870977401733
Bessell time 8 19.332574605941772
Bessell time 9 21.756719827651978
densesvd time 8.838411808013916


100%|██████████| 42439/42439 [01:30<00:00, 466.88it/s]


(50387, 50387)
neg 0.024381160736083984
svd sparse 0.0001838147359872399
sparsesvd time 66.10629916191101
Chebyshev Series -----------------
Bessell time 2 6.396374940872192
Bessell time 3 9.674753427505493
Bessell time 4 12.98150897026062
Bessell time 5 16.314154148101807
Bessell time 6 19.641589641571045
Bessell time 7 22.952343225479126
Bessell time 8 26.247660160064697
Bessell time 9 29.558828592300415
densesvd time 9.002487182617188


100%|██████████| 50387/50387 [01:59<00:00, 420.30it/s]
